# <center> VAI Store - Análise Exploratória de Dados </center>

---

Nesse jupyter notebook iremos analisar os dados de produtos e vendas da VAI Store

## 1. Imports

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as ticker
import importlib

In [ ]:
from pathlib import Path
import sys

sys.path.insert(0, str(Path.cwd().parent / 'src' / 'plots'))
sys.path.insert(0, str(Path.cwd().parent / 'src' / 'utils'))

## 2. Dataset

In [ ]:
produtos = pd.read_csv('../data/raw/produto.csv', index_col = 0)

# Leitura dos dados
produtos.info()
produtos.head()

Sumário das colunas:
* SKU = Código do produto 
* NOME_PRODUTO = Descrição do produto 
* CATEGORIA = Categoria do produto 
* SUBCATEGORIA = Subcategoria do produto

In [ ]:
vendas = pd.read_csv('../data/raw/vendas.csv', index_col = 0)

# Leitura dos dados
vendas.info()
vendas.head()

Sumário das colunas:
* COD_ATEND = Código identificador do atendimento 
* COD_VENDA = Código identificador da venda 
* COD_FILIAL = Indica em qual loja foi feito o atendimento 
* FILIAL = Tipo de filial (Loja de Rua ou de Shopping) 
* DATA_ATEND = Data de atendimento daquele cliente a respeito daquele produto 
* SKU = Código do produto 
* UNID = Indica tipo do produto (granel ou prateleira) 
* QTD_VENDA = Quantidade vendida do produto 
* FATUR_VENDA  = Faturamento 
* CLI_CPF = Código do cliente

In [ ]:
produtos.shape

## 3. Tratamento

### 3.1. Tratamento de valores ausentes

In [ ]:
vendas.isna().sum()

In [ ]:
produtos.isna().sum()

Observa-se que, no dataset de **produtos**, há a presença de valores nulos em todas as colunas. Sendo assim, vamos analisar caso a caso.

#### 3.1.1. CATEGORIA e SUBCATEGORIA

Para essas colunas categóricas, vamos substituir os valores ausentes por um novo tipo: **'Sem categoria'** e **'Sem subcategoria'**.

In [ ]:
produtos['CATEGORIA'] = produtos['CATEGORIA'].fillna('Sem categoria')
produtos['SUBCATEGORIA'] = produtos['SUBCATEGORIA'].fillna('Sem subcategoria')

#### 3.1.2. NOME_PRODUTO

In [ ]:
produtos[produtos['NOME_PRODUTO'].isna()]

Nesse caso, a linha é constituída apenas de valores nulos e, além disso, não há a possibilidade desse produto ter um relacionamento com alguma venda. Sendo assim, vamos simplesmente **remover** essa observação.

In [ ]:
produtos.dropna(inplace=True)

In [ ]:
# Quantidade de valores nulos pós tratamento
produtos.isna().sum()

### 3.2. Conversão dos tipos de dados

In [ ]:
# Conversão de COD_FILIAL e SKU (vendas) para str
vendas['COD_FILIAL'] = vendas['COD_FILIAL'].astype(str)
vendas['SKU'] = vendas['SKU'].astype(str)

# Conversão de SKU (produtos) para str
produtos.index = produtos.index.astype(int).astype(str)

# Conversão de DATA_ATEND para 'date'
vendas['DATA_ATEND'] = pd.to_datetime(vendas['DATA_ATEND'])

### 3.3. Limpeza de strings

Na coluna NOME_PRODUTO, os valores possuem espaços em branco após o último caracter. Por isso, vamos remover essas lacunas.

In [ ]:
produtos['NOME_PRODUTO'] = produtos['NOME_PRODUTO'].str.strip()

In [ ]:
# Nome dos produtos após tratamento
produtos['NOME_PRODUTO']

In [ ]:
vendas.head()

In [ ]:
produtos.head()

## 4. EDA

### 4.1. Análise do desempenho de vendas e faturamento

Primeiro, vamos fazer a análise exploratória dos dados voltado para as **vendas** e **faturamento**. Podemos começar entendendo o faturamento total e a variação dele ao longo dos meses de 2024.

In [ ]:
# Importação das funções de plotagem
import plot_faturamento_filial as pff
import plots_faturamento as pf

# Atualização dos módulos na cache
importlib.reload(pff)
importlib.reload(pf)

In [ ]:
vendas_2024 = vendas.loc[vendas['DATA_ATEND'].dt.year == 2024]

faturamento = vendas_2024['FATUR_VENDA'].sum()

faturamento_formatado = f"R$ {faturamento:,.2f}".replace(",", "X").replace(".", ",").replace("X", ".")
print(f"Faturamento Total de 2024: {faturamento_formatado}")

Agora, podemos analisar a distribuição desse faturamento pelos **meses do ano**.

In [ ]:
pf.faturamento_mensal(vendas)

A análise do faturamento mensal revela um padrão de **sazonalidade**, com os meses de Março, Novembro e Dezembro se destacando como possíveis __outliers__. Sendo assim, vamos criar um array com seus respectivos índices para manipular esses outliers posteriormente.

In [ ]:
meses_outliers = [3, 11, 12]

#### Análise do faturamento por filiais

Agora, vamos analisar a distribuição do faturamento total da loja em suas filiais.

In [ ]:
pff.plot_faturamento_total_filial(vendas)

Fica perceptível que há uma diferença de faturamento entre as filiais, então vamos destrinchar essa receita um pouco mais para chegar no fator motor dessa discrepância.

In [ ]:
pff.plot_faturamento_mensal_filial(vendas)

Podemos ver claramente que os meses que moveram a diferença de faturamento entre as filiais foram, novamento, os outliers observados anteriormente (Março, Novembro e Dezembro). Será que nesses meses houve uma diferença no ticket médio do cliente que influênciou a divergência de receita?

In [ ]:
pff.plot_faturamento_e_ticket_medio_mensal(vendas)

Podemos ver no gráfico acima que o faturamento da loja está correlacionado ao seu ticket médio e, justamente nos meses que o faturamento do Shopping foi bem maior que o faturamento da Rua, o ticket médio entre as filiais foi mais discrepante. Por isso, quero entender quais são os fatores que movem essa diferênça de ticket médio.

In [ ]:
pff.plot_clientes_unicos_mensal_filial(vendas)

Com o número de clientes únicos por loja podemos ver que normalmente não há uma grande diferença entre o número de clientes que compra em cada loja. Contudo, os meses de Março e Novembro foram contra essa máxima, tendo uma quantidade considerável a mais de clientes passando pela loja do Shopping do que pela loja da Rua, talvez sendo um dos fatores que moveu a diferença do faturamento entre as filiais nesses meses.

### 4.2. Análise das vendas de produtos

Agora, vamos entender como o faturamento e quantidade de vendas da loja estão distribuídos entre os produtos e suas categorias/subcategorias. Portanto, vamos plotar as **top** categorias mais vendidas em 2024.

In [ ]:
import merge_datasets as md

importlib.reload(md)

In [ ]:
df_agg = md.merge_datasets(vendas, produtos, 'SKU')

#### Análise da sazonalidade de produtos

Dado as principais categorias vendidas, podemos observar quais as mais vendidas nos meses outliers observados no tópico **4.1.** e analisar se há alguma diferença.

### 4.3. Análise do catálogo de produtos

Agora vamos entender quais as **principais categorias de produtos** vendidos pela VAI Store

In [ ]:
# Importação das funções de plotagem
import plots_catalogo_produtos as pc

importlib.reload(pc)

In [ ]:
# Plotar o Top 10 Categorias
pc.plot_top_10_categorias(produtos)

Após conhecer as principais categorias de produtos vamos analisar as **principais subcategorias**

In [ ]:
# Plotar o Top 10 Subcategorias
pc.plot_top_10_subcategorias(produtos)

Diante as informações acima podemos entender como o pool de produtos da VAI Store está estruturado atualmente

#### Análise da sazonalidade de produtos

Dado as principais categorias vendidas, podemos observar quais as mais vendidas nos meses outliers observados no tópico **4.1.** e analisar se há alguma diferença.

### 4.3 Análise da sazonalidade de produtos

Continuando com o estudo da diferença de faturamento entre as filiais, queremos agora entender como a venda dos produtos está impactando essa discrepância de receita

### 4.4. Análise da qualidade dos dados de cliente

Vamos ver se todas as compras registradas tem o CPF do cliente comprador associado ao produto vendido

In [ ]:
nan_count_CPF = vendas['CLI_CPF'].isna().sum()
print(f"Número de valores ausentes na coluna 'CLI_CPF': {nan_count_CPF}")

Devido a informação de que todos as vendas da loja incluem a vinculação ao CPF do cliente, podemos extrair informações importantes sobre a persona dos compradores da VAI Store

In [ ]:
vendas.head()